### Install and Import dependencies

In [ ]:
!pip install langchain faiss-cpu langchain-community bitsandbytes

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


In [ ]:
VECTOR_PATH = "/content/drive/MyDrive/Colab Notebooks/Tests/Repo-RAG/vectorstore/"

# 🧬 Embedding model (deve essere lo stesso usato per indicizzazione!)
embedding = HuggingFaceEmbeddings(
    model_name="intfloat/e5-large-v2",
    encode_kwargs={"normalize_embeddings": True},
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)


In [ ]:
VECTOR_PATH = "/content/drive/MyDrive/Colab Notebooks/Tests/Repo-RAG/vectorstore"
# 🔁 Carica l'indice
db = FAISS.load_local(VECTOR_PATH, embedding, allow_dangerous_deserialization=True)

In [ ]:
# 🔍 Query dell’utente
query = "query: What machine learning models were used for the virtual screening of CDK5 inhibitors?"
#query = "Was Random forest used for the virtual screening of CDK5?"
k = 3  # Numero di risultati

# 🧠 Retrieval semantico
results = db.similarity_search_with_score(query, k=k)
# 📄 Visualizza i risultati
for i, (doc, score) in enumerate(results):
    print(f"\n🔹 Result {i+1} (score: {score:.4f})")
    print(f"📌 Source: {doc.metadata.get('source', 'N/A')}")
    print(f"📖 Section: {doc.metadata.get('section', 'N/A')}")
    print(f"📝 Content:\n{doc.page_content[:1000]}...")

In [ ]:

model_id = "Qwen/Qwen1.5-7B-Chat"

# Load tokenizer and model (quantized 4-bit with bitsandbytes)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",             
    torch_dtype="auto",
    trust_remote_code=True,
    load_in_4bit=True            
)



In [ ]:

context = "\n\n".join([doc.page_content for doc, _ in results])


pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

context = "Six different types of fingerprints were used: Morgan, RDKit, Layered..."

question = "What machine learning models were used in the screening of CDK5 inhibitors?"

prompt = f"""<|im_start|>system
You are a scientific assistant. Use the context below to answer the question concisely.
<|im_end|>

<|im_start|>user
Context:
{context}

Question: {query}
<|im_end|>
<|im_start|>assistant
"""

response = pipe(prompt, max_new_tokens=300, do_sample=False)[0]['generated_text']
print(response)

